In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

# Launch graph
sess = tf.Session() 
# tensorboard --logdir=./logs/xor_logs
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs/xor_logs_r0_1")
writer.add_graph(sess.graph)  # Show the graph

# Initialize TensorFlow variables
sess.run(tf.global_variables_initializer())

for step in range(10001):
    summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=step)

    if step % 100 == 0:
        print(step, sess.run(cost, feed_dict={
              X: x_data, Y: y_data}), sess.run([W1, W2]))

# Accuracy report
h, c, a = sess.run([hypothesis, predicted, accuracy],
                   feed_dict={X: x_data, Y: y_data})
print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.716429 [array([[ 0.80151927,  0.67873508],
       [-1.21564841, -0.30502006]], dtype=float32), array([[ 1.70941234],
       [ 0.34732175]], dtype=float32)]
100 0.704779 [array([[ 0.63990378,  0.67631429],
       [-1.12019134, -0.3027209 ]], dtype=float32), array([[ 1.69452691],
       [ 0.39068761]], dtype=float32)]
200 0.700612 [array([[ 0.51552969,  0.67110848],
       [-1.0613476 , -0.3064115 ]], dtype=float32), array([[ 1.6394335 ],
       [ 0.39673117]], dtype=float32)]
300 0.698264 [array([[ 0.43573704,  0.66753799],
       [-1.01415718, -0.31230244]], dtype=float32), array([[ 1.59391057],
       [ 0.40022311]], dtype=float32)]
400 0.696817 [array([[ 0.38689446,  0.66524172],
       [-0.97487831, -0.32014695]], dtype=float32), array([[ 1.5548619 ],
       [ 0.40256235]], dtype=float32)]
500 0.695826 [array([[ 0.35913756,  0.66392857],
       [-0.94225407, -0.32974321]], dtype=float32), array([[ 1.52056384],
       [ 0.40440163]], dtype=float32)]
600 0.695078 [array([[ 0.34600